In [1]:
import torch
t = torch.randn(2,2).cuda()
print(t.is_cuda)

csv_path = "/home/shine/PoisoningAttack/dataset/"
csv_name = "features_rf_only_cat.csv"
# csv_name = "adgraph_rf_data_encoded.csv"
save_path = "/home/shine/PoisoningAttack/saved_model/surrogate.pt"


True


In [2]:
import os 
import pandas as pd 
import numpy as np
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, make_scorer
from sklearn.utils import resample
from joblib import dump, load

os.getcwd()
data = pd.read_csv(csv_path + csv_name)
#data = data.sample(frac=1) #row shuffle
# print(type(data))
# print(data.head())

nCar = data.shape[0]
nVar = data.shape[1]
print('nCar: %d' % nCar, 'nVar: %d' % nVar )

X = data.drop("CLASS", axis=1)
y = data["CLASS"]

count_class_0, count_class_1 = data['CLASS'].value_counts()
print("-----")
print("NonAD (before dropping) : {}".format(count_class_0))
print("AD (before dropping) : {}".format(count_class_1))

count_class_1_to_keep = int(count_class_0 * 0.1)

class_1_indices = data[data['CLASS'] == 1].index
random_indices = np.random.choice(class_1_indices, size=count_class_1_to_keep, replace=False)
class_1_sample = data.loc[random_indices]
data_sampled = pd.concat([class_1_sample, data[data['CLASS'] == 0]])

data_sampled = data_sampled.reset_index(drop=True)

X = data_sampled.drop("CLASS", axis=1)
y = data_sampled["CLASS"]

count_class_0, count_class_1 = data_sampled['CLASS'].value_counts()
print("-----")
print("NonAD (after dropping) : {}".format(count_class_0))
print("AD (after dropping) : {}".format(count_class_1))
print("Ratio : {:.0f} : {:.0f}".format(count_class_0 / (count_class_0 + count_class_1) * 100, count_class_1 / (count_class_0 + count_class_1) * 100))


clf = RandomForestClassifier(n_estimators=100, random_state=42)


nCar: 432280 nVar: 293
-----
NonAD (before dropping) : 375130
AD (before dropping) : 57150
-----
NonAD (after dropping) : 375130
AD (after dropping) : 37513
Ratio : 91 : 9


In [3]:
# Define the stratified cross-validation object
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

to_exclude = ["DOMAIN_NAME", "NODE_ID", "FEATURE_KATZ_CENTRALITY", "FEATURE_FIRST_PARENT_KATZ_CENTRALITY", "FEATURE_SECOND_PARENT_KATZ_CENTRALITY"]

accuracy_history = []
precision_history = []
recall_history = []

# For each fold
for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    print("Fold ", fold)
    fold_train_y = y[train_index]
    fold_test_y = y[test_index]
    
    for class_label in np.unique(y):
        train_count = np.count_nonzero(fold_train_y == class_label)
        test_count = np.count_nonzero(fold_test_y == class_label)
        if fold == 0 or fold == 5 or fold == 9:
            print("Class label:", class_label, "train count:", train_count, "test count:", test_count)

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    X_train = X_train.drop(to_exclude, axis=1)
    X_test = X_test.drop(to_exclude, axis=1)

    # Train
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    accuracy_history.append(accuracy_score(y_test, y_pred))
    precision_history.append(precision_score(y_test, y_pred))
    recall_history.append(recall_score(y_test, y_pred))

# Print
print("Average score:", np.mean(accuracy_history))
print("Average precision:", np.mean(precision_history))
print("Average recall:", np.mean(recall_history))

Fold  0
Class label: 0 train count: 337617 test count: 37513
Class label: 1 train count: 33761 test count: 3752
Fold  1
Fold  2
Fold  3
Fold  4
Fold  5
Class label: 0 train count: 337617 test count: 37513
Class label: 1 train count: 33762 test count: 3751
Fold  6
Fold  7
Fold  8
Fold  9
Class label: 0 train count: 337617 test count: 37513
Class label: 1 train count: 33762 test count: 3751
Average score: 0.965941499494444
Average precision: 0.8828439905765098
Average recall: 0.7210825798266162


In [86]:
from sklearn.model_selection import train_test_split

X_sampled, _, y_sampled, _ = train_test_split(X, y, train_size=50000, random_state=42)

prediction = clf.predict(X_sampled)

# create dataframe
X_sampled_df = pd.DataFrame(X_sampled)
prediction_df = pd.DataFrame(prediction, columns=["Prediction"])

X_sampled_df = X_sampled_df.reset_index()

# concate
result_df = pd.concat([X_sampled_df, prediction_df], axis=1)

result_df = result_df.drop("index", axis=1)

# save
result_df.to_csv("/home/shine/PoisoningAttack/dataset/query_50000.csv", index=False)

In [4]:
csv_name_ad = "adgraph_rf_data_encoded_only_ad.csv"
csv_name_pgd = "perturbed_features_pgd.csv"
csv_name_nbackdoor = "perturbed_features_nbackdoor.csv"

data_ad = pd.read_csv(csv_path + csv_name_ad)
data_pgd = pd.read_csv(csv_path + csv_name_pgd)
data_nbackdoor = pd.read_csv(csv_path + csv_name_nbackdoor)

feature_columns_ad = list(data_ad.columns.difference(['CLASS']))
feature_columns_pgd = list(data_pgd.columns.difference(['CLASS']))
feature_columns_nbackdoor = list(data_nbackdoor.columns.difference(['CLASS']))

X_ad = data_ad[feature_columns_ad]
X_pgd = data_pgd[feature_columns_pgd]
X_nbackdoor = data_nbackdoor[feature_columns_nbackdoor]

after_mapping_target_ad = data_ad['CLASS']
after_mapping_target_pgd = data_pgd['CLASS']
after_mapping_target_nbackdoor = data_nbackdoor['CLASS']

y_ad = after_mapping_target_ad
y_pgd = after_mapping_target_pgd
y_nbackdoor = after_mapping_target_nbackdoor

predict_ad = clf.predict(X_ad)
predict_pgd = clf.predict(X_pgd)
predict_nbackdoor = clf.predict(X_nbackdoor)

print(accuracy_score(y_ad, predict_ad))
print(accuracy_score(y_pgd, predict_pgd))
print(accuracy_score(y_nbackdoor, predict_nbackdoor))



/home/shine/anaconda3/envs/adgraph/lib/python3.8/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- feat_0
- feat_1
- feat_10
- feat_11
- feat_12
- ...
Feature names seen at fit time, yet now missing:
- 0
- 1
- 10
- 100
- 101
- ...

  warnings.warn(message, FutureWarning)


ValueError: X has 69 features, but RandomForestClassifier is expecting 280 features as input.